In [54]:
"""
# Title: A_Aug_Replace_Class_Using_HSV_Red_Yellow_colour_threshold.jpynb 
# Include with dissertation

## Purpose:
Stage in processing pipeline to make Green Coast flag sets usable.  
Refines Green Coast flag datasets by cleaning up remnants of dark red/yellow left after bounding box filling.  
Uses HSV thresholds to identify and fill flagged regions with colors sampled from surrounding areas, while visualizing sampling regions.

## Workflow:
1. Uses HSV thresholds to identify red and yellow regions in the image.
2. Restricts mask refinement to areas above or adjacent to bounding boxes for the target class.
3. Samples colors from surrounding regions (left, right, or up) to fill flagged areas.
4. Saves the cleaned images and copies the corresponding YOLO annotation files.

## Parameters:
- `src_folder` (str): Path to the folder containing source images.
- `tgt_folder` (str): Path to save processed images and annotations.
- `annotations_src_folder` (str): Path to the folder containing YOLO annotations.
- `target_class_id` (int): The class ID of the bounding boxes to refine.

## Dependencies:
- Python 3.x
- OpenCV
- NumPy
- Matplotlib

## Notes:
- Input images should be in `.png` format with YOLO `.txt` annotation files.
- Sampling is configured for left, right, or up regions. Uncomment appropriate sections as needed.
"""

import os
import cv2
import numpy as np
import shutil
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set a different font to avoid issues with DejaVuSans
rcParams.update({'font.family': 'sans-serif', 'font.sans-serif': ['Arial']})

def fill_flag_mask(img_path, annotations, target_class_id, output_mask_path=None):
    """
    Processes an image to refine red and yellow flag areas within a new search area based on HSV thresholds
    and fills all detected regions with sampled background colors. It visualizes sampling regions with bounding boxes.

    Args:
        img_path (str): Path to the input image.
        annotations (list): List of YOLO-format annotations for the image.
        target_class_id (int): The class ID for which the mask should be refined within the adjusted search area.
        output_mask_path (str): Path to save the processed image with filled areas.

    Returns:
        img_filled (numpy.ndarray): The processed image.
    """
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found or unable to read: {img_path}")
    
    original_img = img.copy()  # Keep a copy for visualization
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define thresholds for red and yellow in HSV
    lower_red1, upper_red1 = np.array([0, 70, 50]), np.array([10, 255, 255])
    lower_red2, upper_red2 = np.array([160, 70, 50]), np.array([180, 255, 255])
    lower_yellow, upper_yellow = np.array([15, 70, 70]), np.array([40, 255, 255])

    # Generate masks for red and yellow
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Combine the masks
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    combined_mask = cv2.bitwise_or(mask_red, mask_yellow)

    # Refine the mask
    kernel = np.ones((3, 3), np.uint8)
    filled_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    smoothed_mask = cv2.GaussianBlur(filled_mask, (5, 5), sigmaX=1)

    # Locate the bounding box for the target class
    image_height, image_width = img.shape[:2]
    target_boxes = [
        annotation for annotation in annotations
        if int(annotation[0]) == target_class_id
    ]

    if not target_boxes:
        raise ValueError(f"No bounding box found for target class ID {target_class_id}.")

    # Process each target bounding box
    for box in target_boxes:
        class_id, x_center, y_center, width, height = box
        x_min = int((x_center - width / 2) * image_width)
        y_min = int((y_center - height / 2) * image_height)
        x_max = int((x_center + width / 2) * image_width)
        y_max = int((y_center + height / 2) * image_height)

        # Calculate the adjusted search area
        search_height = int(height * image_height)
        search_width = int(width * image_width)

        search_x_min = x_min
        search_x_max = x_max
        search_y_max = y_min + (y_max - y_min) // 2
        search_y_min = max(0, search_y_max - search_height)

        # Restrict the mask to the new search area
        restricted_mask = np.zeros_like(smoothed_mask)
        restricted_mask[search_y_min:search_y_max, search_x_min:search_x_max] = smoothed_mask[search_y_min:search_y_max, search_x_min:search_x_max]

        # Find contours in the restricted mask
        contours, _ = cv2.findContours(restricted_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue

        # Iterate through all detected contours
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            # Adjust sampling to only go left and up or right and up
            surrounding_regions = [
                #(max(0, y-10), y, max(0, x-10), x),  # Left
                #(max(0, y-10), y, x+w, min(image_width, x+w+10)),  # Right
                # Uncomment the next line for "Up" sampling (add this alongside Left or Right as needed)
                (max(0, y-10), y, x, x+w)  # Up
            ]

            sampled_colors = []
            for (r_y_min, r_y_max, r_x_min, r_x_max) in surrounding_regions:
                region = img[r_y_min:r_y_max, r_x_min:r_x_max]
                if region.size > 0 and np.any(region):
                    sampled_colors.append(region.mean(axis=(0, 1)))

                    # Draw bounding box around sampling regions for visualization
                    cv2.rectangle(original_img, (r_x_min, r_y_min), (r_x_max, r_y_max), (0, 255, 0), 2)

            if not sampled_colors:
                print(f"Sampling failed for bounding box: x={x}, y={y}, w={w}, h={h}")
                continue

            # Calculate the average color and fill the region
            fill_color = np.mean(sampled_colors, axis=0).astype(np.uint8)
            img[y:y+h, x:x+w][restricted_mask[y:y+h, x:x+w] > 0] = fill_color

    # Visualize the result
    #plt.figure(figsize=(15, 15))
    #plt.subplot(1, 2, 1)
    #plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    #plt.title("Original Image with Sampling Regions")

    #plt.subplot(1, 2, 2)
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #plt.title("Processed Image with Filled Areas")
    #plt.show()

    # Save the result if output path is provided
    if output_mask_path:
        cv2.imwrite(output_mask_path, img)
    return img
    
def fill_class_by_colour(src_folder, tgt_folder, annotations_src_folder, target_class_id):
    """
    Processes all images in a folder to refine and fill flag areas based on HSV thresholds within the adjusted search area.

    Args:
        src_folder (str): Path to the folder containing source images.
        tgt_folder (str): Path to the folder to save processed images.
        annotations_src_folder (str): Path to the folder containing YOLO annotations.
        target_class_id (int): The class ID for which the mask should be refined within the adjusted search area.
    """
    if not os.path.exists(tgt_folder):
        os.makedirs(tgt_folder)

    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            src_image_path = os.path.normpath(os.path.join(src_folder, file_name))
            annotation_path = os.path.normpath(os.path.join(annotations_src_folder, os.path.splitext(file_name)[0] + ".txt"))
            
            if not os.path.exists(annotation_path):
                print(f"No annotation file for {file_name} in {src_folder}. Skipping.")
                continue

            with open(annotation_path, 'r') as f:
                annotations = [list(map(float, line.split())) for line in f]

            intermediate_mask_path = os.path.normpath(os.path.join(tgt_folder, f"{file_name}"))
            fill_flag_mask(src_image_path, annotations, target_class_id, intermediate_mask_path)

            # Copy the annotation file to the target folder
            target_annotation_path = os.path.normpath(os.path.join(tgt_folder, os.path.basename(annotation_path)))
            shutil.copy(annotation_path, target_annotation_path)
            # print(f"Copied annotation to {target_annotation_path}")

# Run to fill target area with bg colour 
src_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_ClassFill'
tgt_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_HSVFill'
annotations_src_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_ClassFill'
target_class_id = 3  # Class ID to target = 3 Green Cost flag 

fill_class_by_colour(src_folder, tgt_folder, annotations_src_folder, target_class_id)
# Run Stage 1 first - Stage 1: A_Aug_Apply_BG_Fill_For_Flag_Class
